In [1]:
import pandas as pd
import numpy as np

In [34]:
df = pd.read_excel("Hw1-words.xlsx", sheet_name='Proto-words')  # Replace 'Sheet1' with the name of the sheet you want to read
df

,father,mother,one,eight,knee,nose,tongue,dog,egg,smoke,snow,to burn,to bite,to sing,to spit,dry
0,*ićä,*ava,*ikte,*kakteksan,*polwe,*nere,*kele,*pene,*muna,*sawe,*lume,*korpe,*pure,*mura,*śil'ke,*kuśka
1,*ićä,*emä,*ikte,*kakteksan,*polwe,*nenä,*kele,*pene,*muna,*sawe,*lume,*korpe,*pure,*laulu,*śil'ke,*kujwa
2,*ićä,*emä,*egy,*ńalɜ,*térd,*orr,*nyelv,*ämpɜ,*tojás,*pičɜ,*hó,*pilɜ,*harap,*énekel,*köp,*ϑasɜ
3,*ićä,*emä,*ikte,*ńalɜ,*sāns,*nere,*kele,*ämpɜ,*muna,*pičɜ,*tūjt,*pilɜ,*pure,*mura,*śil'ke,*ϑasɜ
4,*ićä,*ava,*ikte,*kakteksan,*polwe,*nere,*jylme,*pene,*muna,*šikš,*lume,*äŋɜ,*pure,*mura,*śil'ke,*kuśka
5,*ićä,*emä,*ikte,*kakteksan,*polwe,*nere,*kele,*pene,*muna,*sawe,*lume,*korpe,*pure,*laulu,*śil'ke,*kujwa


In [61]:
# Concatenate all values as 
all_text_values = pd.concat([df[col] for col in df.columns])

# List unique text values
unique_values = all_text_values.unique()

In [62]:
keys = list('ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklm')
len(keys)
proto_dict = dict(zip(unique_values, keys))
df.replace(proto_dict, inplace=True)

In [63]:
matrix_size = df.shape[0]
NameIndexed = []
for i in range(matrix_size):
    NameIndexed.append("S"+str(i+1))
print("Column and Row names",NameIndexed)

Column and Row names ['S1', 'S2', 'S3', 'S4', 'S5', 'S6']


In [64]:
distance_matrix = pd.DataFrame(index=range(df.shape[0]), columns=range(df.shape[0]))
distance_matrix = pd.DataFrame(distance_matrix, index=NameIndexed)
distance_matrix.columns = NameIndexed

In [66]:
min_value_location = []
min_value = 999999
for i in range (matrix_size):
    for j in range (matrix_size-i,0,-1):
        col=matrix_size-j
        row =i
        row1 = df.iloc[row]
        row2 = df.iloc[col]
        distance = (row1 != row2).sum()
        if min_value>distance and distance!=0:
            min_value = distance
            min_value_location = (row,col)        
#         print(f"Distance between row {row} and {col}", "Distance =", distance) #Diagnostics.
        distance_matrix.iloc[row,col]=distance


### <u>**Distance matrix from Proto words**</u>

In [68]:
distance_matrix

,S1,S2,S3,S4,S5,S6
S1,0,4,15,8,3,3
S2,NaN,0,14,9,7,1
S3,NaN,NaN,0,9,15,14
S4,NaN,NaN,NaN,0,9,8
S5,NaN,NaN,NaN,NaN,0,6
S6,NaN,NaN,NaN,NaN,NaN,0


### Renaming columns as S1..Sn & finding the row/col index of the minimum distance

In [69]:
distance_matrix.columns = NameIndexed
distance_matrix.shape
lowerIndex, upperIndex = min_value_location
print(f"Minimum distance row/col index==> {min_value_location}")

Minimum distance row/col index==> (1, 5)


In [70]:
newRowColName = f"S{lowerIndex+1}{upperIndex+1}"

### Creating a copy of distance matrix for calculation of 2nd step and removing the rows and column associated with minimum row/col distance.

In [221]:
distance_matrix_l1 = distance_matrix.copy()
distance_matrix_l1.drop(distance_matrix_l1.columns[upperIndex], axis=1, inplace=True)
distance_matrix_l1.drop(distance_matrix_l1.columns[lowerIndex], axis=1, inplace=True)

distance_matrix_l1.drop(distance_matrix_l1.index[upperIndex], inplace=True)
distance_matrix_l1.drop(distance_matrix_l1.index[lowerIndex], inplace=True)
distance_matrix_l1

,S1,S3,S4,S5
S1,0,15,8,3
S3,NaN,0,9,15
S4,NaN,NaN,0,9
S5,NaN,NaN,NaN,0


### Adding row and col at index 0,0 with with the row/col name.

In [222]:
newDF = pd.DataFrame(rowCombo, index=[0])
distance_matrix_l1 = pd.concat([newDF, distance_matrix_l1])
rowCombo  = distance_matrix_l1.iloc[0]
rowArray  = np.asarray(rowCombo)

In [223]:
distance_matrix_l1

,0,S1,S3,S4,S5
0,NaN,NaN,NaN,NaN,NaN
S1,NaN,0,15,8,3
S3,NaN,NaN,0,9,15
S4,NaN,NaN,NaN,0,9
S5,NaN,NaN,NaN,NaN,0


In [226]:
rowArray[:] = np.nan

Insert column at index=0 and rename column

In [227]:
distance_matrix_l1.insert(0, f"S{lowerIndex+1}{upperIndex+1}", rowArray)


distance_matrix_l1

,S26,0,S1,S3,S4,S5
0,NaN,NaN,NaN,NaN,NaN,NaN
S1,NaN,NaN,0,15,8,3
S3,NaN,NaN,NaN,0,9,15
S4,NaN,NaN,NaN,NaN,0,9
S5,NaN,NaN,NaN,NaN,NaN,0


In [228]:
indexNames = distance_matrix_l1.index.tolist()
indexNames[0]= f"S{lowerIndex+1}{upperIndex+1}"
distance_matrix_l1.index= indexNames

#### Renamed row and column names for minimum distance combo name.

In [230]:
distance_matrix_l1

,S26,0,S1,S3,S4,S5
S26,NaN,NaN,NaN,NaN,NaN,NaN
S1,NaN,NaN,0,15,8,3
S3,NaN,NaN,NaN,0,9,15
S4,NaN,NaN,NaN,NaN,0,9
S5,NaN,NaN,NaN,NaN,NaN,0
